# 5章 文章の穴埋め

- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

BERTは文章の一部を`[MASK]`トークンに変換したものを読み込み、`[MASK]`トークンに当てはまるトークンが何かを予測するかたちで事前学習を行っている。同様に、BERTは一部文章の穴埋めを行うことができる。

## 5.1 パッケージのインストール

文章の穴埋めには、`transformers.BertForMaskedLM`クラスを用いる。

In [4]:
# 5-2

import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

## 5.2 BERTを用いた文章の穴埋め

トークナイザと穴埋めモデルをロードする。<br>
ロードにはそれぞれのクラスの`from_pretrained()`メソッドを使用する。

In [7]:
# 5-3

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<br>

モデルをGPUメモリ上で動作させるには、以下を実行しておく

In [8]:
# model = model.cuda()

<br>

今回は、
```
今日は[MASK]へ行く。
```
という文章の`[MASK]`トークンの穴埋めをする。まずは文章をトークン化して確認してみる。

In [9]:
# 5-4

text = "今日は[MASK]へ行く。"
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


<br>

`transformers.BertForMaskedLM`に符号化した文章を渡す。

In [11]:
# 5-5
# 符号化した文章のみほしい、系列長を揃える必要がない
# のでtokenizer()関数は必要なし
input_ids = tokenizer.encode(text, return_tensors='pt')
# input_ids = input_ids.cuda()

# BERTに入力する
with torch.no_grad():
    output = model(input_ids=input_ids) # 可変長引数を用いない場合、引数を指定する
    scores = output.logits

In [14]:
scores.size()

torch.Size([1, 8, 32000])

出力`transformers.modeling_outputs.MaskedLMOutput`のクラス変数`output.logits`として、語彙に含まれる各トークンの分類スコアを表すテンソル`scores`を得られる。<br>
具体的には、トークン化した文章 + 文頭トークン`[CLS]` + 文末トークン`[SEP]`の8トークンに対し、`WordPiece`が持つ32000語彙のそれぞれに当てはまる確率を計算する。

モデルが穴埋めした結果を見ていく。

In [50]:
# 5-6
# [MASK]トークンの位置
mask_position = input_ids[0].tolist().index(4)

# スコアが最もいいトークンのID(インデックス)を取り出す
id_best = scores[0, mask_position].argmax().item()

# トークンIDをトークンに置き換える
token_best = tokenizer.convert_ids_to_tokens(id_best)

# [MASK]をトークンに置き換える
text.replace("[MASK]", token_best)

'今日は東京へ行く。'

In [ ]:
# 5-7
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    上位何位まで使うかは、num_topkで指定。
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める。
    mask_position = input_ids[0].tolist().index(4) 
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

In [ ]:
# 5-8
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する。
    """
    # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える。
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

In [ ]:
# 5-9
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

In [ ]:
# 5-10
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

In [ ]:
# 5-11
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')